# Starting our Docker Environment

In [ ]:
!./gradlew --console=plain tasks --group docker -q

In [ ]:
!./gradlew --console=plain -q composeUp

In [ ]:
ksql = "/usr/local/bin/ksql"
!echo "LIST TOPICS;" | "$ksql"

# The Developer Experience
### Create a *registration* stream `CLICKSTREAM`:

In [ ]:
sql = (
    "CREATE STREAM clickstream "
    "(_time bigint, time varchar, ip varchar, request varchar, status int, userid int, bytes bigint, agent varchar) "
    "with (kafka_topic = 'clickstream', value_format = 'json');"
)
!echo "$sql" | "$ksql"

### Create table EVENTS_PER_MIN

Now, let's create create another table called `EVENTS_PER_MIN` that uses `CLICKSTREAM`. This is a *persistent query*... it selects data from an existing KSQL stream or table, creates or uses an underlying Kafka topic as a target and initializes the streaming process to transform and persist data to that topic:

In [ ]:
sql = (
    "CREATE TABLE events_per_min AS SELECT userid, count(*) AS events "
    "FROM clickstream window "
    "TUMBLING (size 60 second) GROUP BY userid;"
)
!echo "$sql" | "$ksql"

### Iterating on our Pipeline
Now, I want to make some changes to `CLICKSTREAM`. I'll just drop it right?
### Drop stream CLICKSTREAM

In [ ]:
sql = (
    "DROP STREAM clickstream; "
)
!echo "$sql" | "$ksql"

### Terminate our auto-incrementing query id

In [ ]:
sql = (
    "TERMINATE QUERY CTAS_EVENTS_PER_MIN_0; "
)
!echo "$sql" | "$ksql"

### Now drop CLICKSTREAM again

In [ ]:
sql = (
    "DROP STREAM clickstream; "
)
!echo "$sql" | "$ksql"

### Is this workable long-term?
Wouldn't it be great if this could be automated?

# The `gradle-confluent` Plugin

[GitHub Repository](https://github.com/RedPillAnalytics/gradle-confluent)

[Gradle Plugin Portal](https://plugins.gradle.org/plugin/com.redpillanalytics.gradle-confluent)


Our `build.gradle` file:

In [ ]:
!./gradlew ksql:tasks --group confluent

### The `pipelineExecute` task

In [ ]:
!./gradlew --console=plain help --task ksql:pipelineExecute

### Executing the `01-clickstream` pipeline

In [ ]:
!./gradlew --console=plain ksql:pipelineExecute --pipeline-dir 01-clickstream

In [ ]:
!./gradlew --console=plain ksql:pipelineExecute -i

### Building KSQL artifacts

In [ ]:
!./gradlew --console=plain ksql:build ksql:publish

In [ ]:
!zipinfo ~/.m2/repository/com/redpillanalytics/ksql-pipeline/1.0.0/ksql-pipeline-1.0.0.zip

In [ ]:
!cat ksql/build/pipeline/ksql-script.sql

### Deploying KSQL artifacts

In [ ]:
!./gradlew --console=plain ksql:deploy

### KSQL Directives

We needed a way to control some of the auto-generated SQL statements.

For instance... we wanted to add `... DELETE TOPIC` granularly to some of our KSQL scripts.

We introduced *directives*... smart comments beginning with `--@`.

In [ ]:
!./gradlew --console=plain ksql:pipelineExecute --pipeline-dir 01-clickstream -i

# KSQL User Defined Functions

### Create `decode()` function for a `CASE`-like experience

### And here is a snippet from our *test specification* using the [Spock Framework](http://spockframework.org/):

### And here is our `build.gradle` file for the `functions` subproject:

### Let's build and publish our UDF artifact, and see how many items are in it:

In [ ]:
!./gradlew --console=plain functions:build functions:publish
!zipinfo -h ~/.m2/repository/com/redpillanalytics/functions/1.0.0/functions-1.0.0.jar

In [ ]:
docker = "/usr/local/bin/docker"
!"$docker" cp ~/.m2/repository/com/redpillanalytics/functions/1.0.0/functions-1.0.0.jar ksql-server:/etc/ksql-server/ext

In [ ]:
!echo "LIST TOPICS;" | "$ksql"

In [ ]:
!echo "DESCRIBE FUNCTION DECODE;" | "$ksql"

In [ ]:
!./gradlew --console=plain ksql:deploy

In [ ]:
sql = (
    "select definition, decode(definition, "
    "'Proxy authentication required','Bad', "
    "'Page not found','Bad', "
    "'Redirect','Good', "
    "'Unknown') label "
    "from enriched_error_codes limit 5;"
)
!echo "$sql" | "$ksql"

# Stopping our Docker environment

In [ ]:
!./gradlew --console=plain -q composeDown